In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import java.util.Properties

In [3]:
object Conexao {
  def criarSessaoSpark(): SparkSession = {
    val spark = SparkSession.builder()
      .appName("Gerar Massa de Dados")
      .master("spark://spark-master:7077")
      .config("spark.jars", "/opt/bitnami/spark/jars/mysql-connector-j-8.0.33.jar")
      .getOrCreate()
    spark
  }

  def propriedadesJDBC(): Properties = {
    val jdbcUrl = "jdbc:mysql://mysql:3306/desafio" 
    val jdbcProperties = new Properties()
    jdbcProperties.setProperty("user", "sparkuser")
    jdbcProperties.setProperty("password", "sparkpass")
    jdbcProperties.setProperty("driver", "com.mysql.cj.jdbc.Driver")
    jdbcProperties
  }

  val jdbcUrl: String = "jdbc:mysql://mysql:3306/desafio"
}

defined object Conexao


In [4]:
object Associado {
  def criarAssociados(spark: SparkSession) = {
    import spark.implicits._

    val nomes = Seq("Wilian", "Natalia", "Gustavo", "Eder", "Wallace", "Wendy", "Caio", "Joao")
    val sobrenomes = Seq("Gomes", "Santana", "Mateo", "Murada", "Vieira", "Cruz", "Monteiro", "Lanzana")

    val associados = spark.range(1, 51).select(
      col("id"),
      element_at(array(nomes.map(lit): _*), (floor(rand() * nomes.length) + 1).cast("int")).alias("nome"),
      element_at(array(sobrenomes.map(lit): _*), (floor(rand() * sobrenomes.length) + 1).cast("int")).alias("sobrenome"),
      (floor(rand() * 50) + 18).alias("idade"),
      concat_ws("", lower(col("nome")), lower(col("sobrenome")), col("id").cast("string"), lit("@email.com")).alias("email")
    )
    println("DataFrame 'associado' criado:")
    associados.show(5)
    associados
  }
}

defined object Associado


In [5]:
object Conta {
  def criarContas(spark: SparkSession, associados: DataFrame) = {
    import spark.implicits._

    val contas = associados.select(
      col("id"),
      when(rand() > 0.5, "Corrente").otherwise("Poupança").alias("tipo_conta"),
      current_timestamp().alias("data_criacao"),
      col("id").alias("id_associado")
    )
    println("DataFrame 'conta' criado:")
    contas.show(5)
    contas
  }
}

defined object Conta


In [6]:
object Cartao {
  def criarCartoes(spark: SparkSession, associados: DataFrame) = {
    import spark.implicits._

    val cartoes = associados.select(
      col("id"),
      (floor(rand() * 90000000) + 10000000).alias("num_cartao"),
      concat_ws(" ", col("nome"), col("sobrenome")).alias("nom_impresso"),
      col("id").alias("id_conta"),
      col("id").alias("id_associado")
    )
    println("DataFrame 'cartao' criado:")
    cartoes.show(5)
    cartoes
  }
}

defined object Cartao


In [7]:
object Movimento {
  def criarMovimentos(spark: SparkSession, cartoes: DataFrame) = {
    import spark.implicits._

    val movimentos = cartoes.select(
      col("id"),
      (rand() * 1000).cast("decimal(10,2)").alias("vlr_transacao"),
      element_at(array(lit("Compra"), lit("Saque"), lit("Depósito")), (floor(rand() * 3) + 1).cast("int")).alias("des_transacao"),
      current_timestamp().alias("data_movimento"),
      col("id").alias("id_cartao")
    )
    println("DataFrame 'movimento' criado:")
    movimentos.show(5)
    movimentos
  }
}

defined object Movimento


In [8]:
val spark = Conexao.criarSessaoSpark()

spark = org.apache.spark.sql.SparkSession@2da9e122


org.apache.spark.sql.SparkSession@2da9e122

In [9]:
val associados = Associado.criarAssociados(spark)

DataFrame 'associado' criado:
+---+-------+---------+-----+--------------------+
| id|   nome|sobrenome|idade|               email|
+---+-------+---------+-----+--------------------+
|  1|   Eder|  Santana|   43|edersantana1@emai...|
|  2|Gustavo|   Vieira|   22|gustavovieira2@em...|
|  3|Wallace|    Mateo|   53|wallacemateo3@ema...|
|  4|   Caio| Monteiro|   54|caiomonteiro4@ema...|
|  5|Wallace|    Gomes|   54|wallacegomes5@ema...|
+---+-------+---------+-----+--------------------+
only showing top 5 rows



associados = [id: bigint, nome: string ... 3 more fields]


[id: bigint, nome: string ... 3 more fields]

In [10]:
val contas = Conta.criarContas(spark, associados)

DataFrame 'conta' criado:


contas = [id: bigint, tipo_conta: string ... 2 more fields]


+---+----------+--------------------+------------+
| id|tipo_conta|        data_criacao|id_associado|
+---+----------+--------------------+------------+
|  1|  Corrente|2025-03-29 03:58:...|           1|
|  2|  Corrente|2025-03-29 03:58:...|           2|
|  3|  Corrente|2025-03-29 03:58:...|           3|
|  4|  Poupança|2025-03-29 03:58:...|           4|
|  5|  Poupança|2025-03-29 03:58:...|           5|
+---+----------+--------------------+------------+
only showing top 5 rows



[id: bigint, tipo_conta: string ... 2 more fields]

In [11]:
val cartoes = Cartao.criarCartoes(spark, associados)

DataFrame 'cartao' criado:


cartoes = [id: bigint, num_cartao: bigint ... 3 more fields]


+---+----------+--------------+--------+------------+
| id|num_cartao|  nom_impresso|id_conta|id_associado|
+---+----------+--------------+--------+------------+
|  1|  20322634|  Eder Santana|       1|           1|
|  2|  23994831|Gustavo Vieira|       2|           2|
|  3|  92517386| Wallace Mateo|       3|           3|
|  4|  30618399| Caio Monteiro|       4|           4|
|  5|  97579624| Wallace Gomes|       5|           5|
+---+----------+--------------+--------+------------+
only showing top 5 rows



[id: bigint, num_cartao: bigint ... 3 more fields]

In [12]:
val movimentos = Movimento.criarMovimentos(spark, cartoes)

DataFrame 'movimento' criado:
+---+-------------+-------------+--------------------+---------+
| id|vlr_transacao|des_transacao|      data_movimento|id_cartao|
+---+-------------+-------------+--------------------+---------+
|  1|       500.91|        Saque|2025-03-29 03:58:...|        1|
|  2|       541.25|     Depósito|2025-03-29 03:58:...|        2|
|  3|       173.34|        Saque|2025-03-29 03:58:...|        3|
|  4|       485.14|     Depósito|2025-03-29 03:58:...|        4|
|  5|        53.18|       Compra|2025-03-29 03:58:...|        5|
+---+-------------+-------------+--------------------+---------+
only showing top 5 rows



movimentos = [id: bigint, vlr_transacao: decimal(10,2) ... 3 more fields]


[id: bigint, vlr_transacao: decimal(10,2) ... 3 more fields]

In [13]:
val jdbcProperties = Conexao.propriedadesJDBC()


jdbcProperties = {password=sparkpass, driver=com.mysql.cj.jdbc.Driver, user=sparkuser}


{password=sparkpass, driver=com.mysql.cj.jdbc.Driver, user=sparkuser}

In [14]:
val jdbcUrl = Conexao.jdbcUrl

jdbcUrl = jdbc:mysql://mysql:3306/desafio


jdbc:mysql://mysql:3306/desafio

In [15]:
println("Gravando os dados no MySQL...")
associados.write.mode("append").jdbc(jdbcUrl, "associado", jdbcProperties)
contas.write.mode("append").jdbc(jdbcUrl, "conta", jdbcProperties)
cartoes.write.mode("append").jdbc(jdbcUrl, "cartao", jdbcProperties)
movimentos.write.mode("append").jdbc(jdbcUrl, "movimento", jdbcProperties)
println("Tudo gerado e inserido com sucesso!")

Gravando os dados no MySQL...
Tudo gerado e inserido com sucesso!


In [16]:
spark.stop()